In [1]:
import pandas as pd
import os
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
sc.master

'local[*]'

In [5]:
import re 
from operator import add

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

df = spark.read.csv('/ca2/ProjectTweets.csv',header=True,sep="|")

print(df.collect())

IOPub data rate exceeded.                                                       
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
# spark is from the previous example.
sc = spark.sparkContext

# A CSV dataset is pointed to by path.

path = "/ca2/ProjectTweets.csv"
TweetsDF = spark.read.csv(path)

In [39]:
TweetsDF.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



In [40]:
# Creates a temporary view using the DataFrame
TweetsDF.createOrReplaceTempView("tweets")

In [41]:
# SQL statements can be run by using the sql methods provided by spark
tweetsDF = spark.sql("SELECT _c0 as Row, _c1 as IDs, _c2 as Date, _c3 as Query, _c4 as User, _c5 as Tweet FROM tweets")
tweetsDF.show()

+---+----------+--------------------+--------+---------------+--------------------+
|Row|       IDs|                Date|   Query|           User|               Tweet|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [98]:
#df = spark.read.load("examples/src/main/resources/users.parquet")
tweetsDF.select("Tweet").write.save("Tweet.parquet")

In [99]:
location

NameError: name 'location' is not defined

In [42]:
userDF = spark.sql("SELECT _c4 as User FROM tweets")
userDF.show()

+---------------+
|           User|
+---------------+
|_TheSpecialOne_|
|  scotthamilton|
|       mattycus|
|        ElleCTF|
|         Karoli|
|       joy_wolf|
|        mybirch|
|           coZZ|
|2Hood4Hollywood|
|        mimismo|
| erinx3leannexo|
|   pardonlauren|
|           TLeC|
|robrobbierobert|
|    bayofwolves|
|     HairByJess|
| lovesongwriter|
|       armotley|
|     starkissed|
|      gi_gi_bee|
+---------------+
only showing top 20 rows



In [43]:
textDF = spark.sql("SELECT _c5 as Tweets FROM tweets")
textDF.show()

+--------------------+
|              Tweets|
+--------------------+
|@switchfoot http:...|
|is upset that he ...|
|@Kenichan I dived...|
|my whole body fee...|
|@nationwideclass ...|
|@Kwesidei not the...|
|         Need a hug |
|@LOLTrish hey  lo...|
|@Tatiana_K nope t...|
|@twittera que me ...|
|spring break in p...|
|I just re-pierced...|
|@caregiving I cou...|
|@octolinz16 It it...|
|@smarrison i woul...|
|@iamjazzyfizzle I...|
|Hollis' death sce...|
|about to file taxes |
|@LettyA ahh ive a...|
|@FakerPattyPattz ...|
+--------------------+
only showing top 20 rows



In [45]:
#Looking for duplicates
print('Count of rows: {0}'.format(textDF.count()))
print('Count of distinct rows: {0}'.format(textDF.distinct().count()))

Count of rows: 1600000


Count of distinct rows: 1581466


In [64]:
from pyspark.sql.types import IntegerType, DateType, LongType, TimestampType, StringType, FloatType, DoubleType, DecimalType, StructType, StructField

In [86]:
tweet = sc.textFile('/ca2/ProjectTweets.csv')
header = tweet.first()

tweet = tweet \
    .filter(lambda row: row != header) \
    .map(lambda row: [int(elem) for elem in row.split(',')])

2023-11-05 00:47:53,234 WARN impl.BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:2939)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:821)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:746)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:379)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:644)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:575)
	

In [87]:
pt = StructType([
    StructField("row", IntegerType(),True),
    StructField("id", IntegerType(),True),
    StructField("date", DateType(),True),
    StructField("query", StringType(),True),
    StructField("user", StringType(),True),
    StructField("text", StringType(),True) 
    
])
schema = typ.StructType(pt)

In [88]:
tweets_df = spark.read.format('csv') \
.schema(pt) \
.load(path)

In [89]:
display (tweets_df)

DataFrame[row: int, id: int, date: date, query: string, user: string, text: string]

In [90]:
fraud_df = spark.createDataFrame(tweet,pt)

In [91]:
fraud_df.printSchema()

root
 |-- row: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- query: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [92]:
tweets_df.printSchema()

root
 |-- row: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- query: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [97]:
QueryDF.show()

NameError: name 'QueryDF' is not defined

In [96]:
userDF.show()

+---------------+
|           User|
+---------------+
|_TheSpecialOne_|
|  scotthamilton|
|       mattycus|
|        ElleCTF|
|         Karoli|
|       joy_wolf|
|        mybirch|
|           coZZ|
|2Hood4Hollywood|
|        mimismo|
| erinx3leannexo|
|   pardonlauren|
|           TLeC|
|robrobbierobert|
|    bayofwolves|
|     HairByJess|
| lovesongwriter|
|       armotley|
|     starkissed|
|      gi_gi_bee|
+---------------+
only showing top 20 rows

